In [1]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import dash_daq as daq
from dash.exceptions import PreventUpdate

import socket
from collections import deque

X = deque(maxlen=20)
X.append(1)
Y1 = deque(maxlen=400)
Y1.append(1)
Y2 = deque(maxlen=400)
Y2.append(2)
Y3 = deque(maxlen=400)
Y3.append(3)

x_data = [50]
y_data = [50]
z_data = [50]
mytemp = []
mygauge = []
mytank = []

def read_address(address):
    
    # Send the "read*4" command to the server every second
    command = "read*" + str(address)
    command_obj = command.encode('ascii')
    client_socket.sendall(command_obj)

    # Read data from the server
    response = client_socket.recv(1024)
    response_str = response.decode("utf-8")
    response_int = int(response_str)
    
    return response_int

# Define server details
SERVER_IP = '127.0.0.1'
SERVER_PORT = 7546

# Initialize the Dash application
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.H1("SERVER-CLIENT CONNECTION: STM32", style={"textAlign": "center", "color": "#fec036","margin": "0","background-color": "#2b2b2b"}),
        html.Div(
            id='blue-div',  
            children=[
                html.Div(
                    id='small-div-1',  
                    children=[
                        html.Button('Button 1', id='button-1', n_clicks=0, style={"margin": "20px", "width": "200px",
                                                                                    "height": "80", "borderWidth": "5px",
                                                                                    "fontWeight": "bold", "color": "#fec036",
                                                                                    "borderColor": "#fec036"}),
                        html.Button('Button 2', id='button-2', n_clicks=0, style={"margin": "20px", "width": "200px",
                                                                                    "height": "80", "borderWidth": "5px",
                                                                                    "fontWeight": "bold", "color": "#fec036",
                                                                                    "borderColor": "#fec036"}),
                        html.Div(id='output-state')
                    ],
                    style={
                        "width": "200px",  
                        "height": "200px",  
                        "background-color": "#2b2b2b",  
                        "margin": "20px 0"  
                    }
                ),
                html.Div(
                    id='small-div-2',  
                    children=[
                        html.Label("Enter Server Command:",style={'color': "#fec036"}),
                        dcc.Input(id='command-input', type='text', value='',style={'color': "#fec036"}),
                        html.Button('Send', id='submit-command', n_clicks=0,style={ 
                                                                                   "borderWidth": "5px",
                                                                                  "fontWeight": "bold", "color": "#fec036",
                                                                                  "borderColor": "#fec036"}),
                        html.Div(id='command-output',style={'color': "#fec036"}),
                        html.Div(id='server-response',style={'color': "#fec036"})
                    ],
                    style={
                        "width": "200px",  
                        "height": "200px",  
                        "background-color": "#2b2b2b",  
                        "margin": "20px 0"  
                    }
                ),
                html.Div(
                    id='small-div-21',  
                    children=[
                        daq.Tank(
                            id='my-tank-1',
                            value=150,
                            min=0,
                            max=200,
                            color="#fec036",
                            style={'margin-left': '50px'}
                        ),
                        daq.Slider(
                            id='my-daq-slider-ex-1',
                            value=90,
                            color="#fec036",
                        ),
                    ],
                    style={
                        "width": "200px",  
                        "height": "200px",  
                        "background-color": "#2b2b2b",  
                        "margin": "20px 0",  
                        "display": "block"  
                    }
                )
            ],
            style={
                "width": "30%",  
                "height": "800px",  
                "background-color": "#2b2b2b",  
                "display": "inline-block",  
                "margin": "0",  
                "float": "left"  
            }
        ),
        html.Div(
            id='red-div',  
            children=[
                html.Div(
                    id="small-div-3",
                    children=[
                        
                        dcc.Graph(id='live-graph', animate=True,  style={"width": "800px", "height": "400px"}),
                        dcc.Interval(
                            id='graph-update',
                            interval=1 * 1000,
                        )
                    ],
                    style={
                        "width": "800px",  
                        "height": "400px",  
                        "background-color": "#2b2b2b",  
                        "display": "block",
                        "margin": "20px 0"  
                    }
                ),

                html.Div(
                    id="small-div-6",
                    children=[
                        daq.Gauge(
                            id='my-gauge',
                            label='Speed',
                            value=270,
                            min=0,
                            max=360,
                            size=200,
                            color="#fec036",
                        )
                    ],
                    style={
                        "width": "400px",  
                        "height": "200px",  
                        "background-color": "#2b2b2b",  

                        "display": "inline-block",
                        "margin": "20px 0"  
                    }
                ),
                html.Div(
                    id="small-div-7",
                    children=[
                        daq.Thermometer(
                            id='mythermo',
                            label='thermometer',
                            min=0,
                            max=105,
                            value=100,
                            color="#fec036",

                            units="C"
                        )
                       
                    ],
                    style={
                        "width": "400px",  
                        "height": "200px",  
                        "background-color": "#2b2b2b",  
                        "display": "inline-block",
                        "margin": "20px 0"  
                    }
                ), 
                
            ],
            style={
                "width": "70%",  
                "height": "800px",  
                "background-color": "#2b2b2b",  
                "display": "inline-block",  
                "margin": "0",  
                "float": "left"  
            }
        )
    ], style={"background-color": "#000000","margin": "0"}),
    
    html.Div(
    id="small-div-85",
    children=[
        
        
    ],
    style={
        "width": "815.12",  
        "height": "200px",  
        "background-color": "red",  
        "margin": "auto",
        "marginTop": "20px",
        "padding": "20px",
        "overflow": "auto",  # Tablonun aşırı taşma durumunda kaydırma çubuğu ekleyelim
        "color": "#ffffff"  # Yazı rengini beyaza ayarlayalım
    }
)
], style={"margin": "0"})

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((SERVER_IP, SERVER_PORT))


# Buton tıklamalarını izleyen geri çağrı işlevi
@app.callback(
    Output('output-state', 'children'),
    [Input('button-1', 'n_clicks'),
     Input('button-2', 'n_clicks')],
)
def update_output(button_1_clicks, button_2_clicks):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    command = None

    if 'button-1' in changed_id:
        command = "write*1"
    elif 'button-2' in changed_id:
        command = "write*2"

    if command:
        client_socket.sendall(command.encode())
        print("Command sent:", command)  # Komutun doğru bir şekilde gönderildiğini kontrol etmek için print ekledik.

    return ''  # Sadece komut gönderimini sağlamak için, sayfada bir çıktı göstermiyoruz.


# Grafik güncellemesi için geri çağrı işlevi
@app.callback(
    Output('live-graph', 'figure'),
    [Input('graph-update', 'n_intervals')]
)
def update_graph_scatter(n):
    # Her saniyede bir "read*4" komutunu sunucuya gönder
    client_socket.sendall(b"read*4")

    # Sunucudan veri oku
    response = client_socket.recv(1024)
    response_str = response.decode("utf-8")
    response_int = int(response_str)

    # Sunucudan veri oku
    client_socket.sendall(b"read*5")
    response1 = client_socket.recv(1024)
    response_str1 = response1.decode("utf-8")
    response_int1 = int(response_str1)

    # Sunucudan veri oku
    client_socket.sendall(b"read*6")
    response2 = client_socket.recv(1024)
    response_str2 = response2.decode("utf-8")
    response_int2 = int(response_str2)

    # Sunucudan veri oku ve mythermo değeri olarak kullan
    client_socket.sendall(b"read*7")
    response3 = client_socket.recv(1024)
    response_str3 = response3.decode("utf-8")
    mythermo_value = int(response_str3)

    mytemp.append(mythermo_value)

    client_socket.sendall(b"read*8")
    response4 = client_socket.recv(1024)
    response_str4 = response4.decode("utf-8")
    mygauge_value = int(response_str4)

    mygauge.append(mygauge_value)

    client_socket.sendall(b"read*9")
    response5 = client_socket.recv(1024)
    response_str5 = response5.decode("utf-8")
    mytank_value = int(response_str5)

    mytank.append(mytank_value)

    x_data.append(response_int)
    x_data_1 = x_data.pop()
    y_data.append(response_int1)
    y_data_1 = y_data.pop()
    z_data.append(response_int2)
    z_data_1 = z_data.pop()

    # Sunucudan gelen veriyi x_data listesine ekle
    X.append(X[-1] + 1)
    Y1.append(int(x_data_1))
    Y2.append(int(y_data_1))
    Y3.append(int(z_data_1))

    # Scatter plot oluştur
    trace1 = go.Scatter(
        x=list(X),
        y=list(Y1),
        name='temperature_value',
        mode='lines+markers'
    )
    trace2 = go.Scatter(
        x=list(X),
        y=list(Y2),
        name='speed_value',
        mode='lines+markers'
    )
    trace3 = go.Scatter(
        x=list(X),
        y=list(Y3),
        name='voltage_value',
        mode='lines+markers'
    )

    # Grafik düzenini oluştur
    layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                       yaxis=dict(range=[min(Y1 + Y2 + Y3), max(Y1 + Y2 + Y3)]))

    return {'data': [trace1, trace2, trace3], 'layout': layout}


@app.callback(
    Output('mythermo', 'value'),  # Güncellenecek bileşen ve özelliği
    [Input('graph-update', 'n_intervals')]  # Tetikleyici
)
def update_thermo_value(n):
    if mytemp:
        new_value = mytemp.pop()  # mytemp listesindeki son elemanı çıkar ve kullan
        return new_value
    else:
        return 100  # Varsayılan bir değer döndür, eğer mytemp listesi boşsa


@app.callback(
    Output('my-gauge', 'value'),  # Güncellenecek bileşen ve özelliği
    [Input('graph-update', 'n_intervals')]  # Tetikleyici
)
def update_mygauge_value(n):
    if mygauge:
        new_value2 = mygauge.pop()  # mytemp listesindeki son elemanı çıkar ve kullan
        return new_value2
    else:
        return 100  # Varsayılan bir değer döndür, eğer mygauge listesi boşsa


@app.callback(
    Output('my-tank-1', 'value'),  # Güncellenecek bileşen ve özelliği
    [Input('graph-update', 'n_intervals')]  # Tetikleyici
)
def update_mytank_value(n):
    if mytank:
        new_value3 = mytank.pop()  # mytank listesindeki son elemanı çıkar ve kullan
        return new_value3
    else:
        return 100  # Varsayılan bir değer döndür, eğer mytank listesi boşsa


@app.callback(
    [Output('command-output', 'children'),
     Output('server-response', 'children')],
    [Input('submit-command', 'n_clicks')],
    [State('command-input', 'value')]
)
def send_command_to_server(n_clicks, command):
    if n_clicks > 0:
        if command:
            client_socket.sendall(command.encode())
            response = client_socket.recv(1024)
            response_str = response.decode("utf-8")
            return (f"Komut gönderildi: {command}", f"Sunucu cevabı: {response_str}")
        else:
            return ("Lütfen bir komut girin.", None)
    else:
        raise PreventUpdate


if __name__ == '__main__':
    app.run(port=8025)


Dash is running on http://127.0.0.1:8025/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8025/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2024 00:36:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:36:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25

127.0.0.1 - - [25/Apr/2024 00:37:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 00:37:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [2]:
%tb

No traceback available to show.
